In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
from glob import glob
from tqdm import tqdm
from os.path import expanduser, join, basename, dirname
import xarray as xr
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from shutil import copy
from sklearn.model_selection import StratifiedKFold
import torch
from tempfile import TemporaryDirectory

from albk.data.utils import idx_to_locate
use_disjoint_files = False


import torch
import torch.nn as nn

from glob import glob
from os.path import expanduser, join, basename, dirname
import xarray as xr
import numpy as np
from tqdm import tqdm
import pandas as pd
from joblib import Parallel, delayed
from itertools import product
import torch
import torch.nn as nn
from glob import glob
from os.path import expanduser, join, basename, dirname
import xarray as xr
import numpy as np
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
from itertools import product
from torch.utils.data import TensorDataset, DataLoader
from astra.torch.models import EfficientNetClassifier,EfficientNet_B0_Weights
from astra.torch.utils import train_fn

import torchvision.models as models
from astra.torch.metrics import accuracy_score, f1_score, precision_score, recall_score,classification_report

In [19]:
# images_path = expanduser("/home/rishabh.mondal/bkdb/statewise/West_Bengal/")
images_path = expanduser('/home/rishabh.mondal/bkdb/districtwise/jhajhar/')
zarr_files = [f for f in os.listdir(images_path) if f.endswith(".zarr")]


In [20]:
files = []
for zarr_file in zarr_files:
    image_path = join(images_path, zarr_file)
    files.append(image_path)

print(len(files))
print(files[:5])

1713
['/home/rishabh.mondal/bkdb/districtwise/jhajhar/28.60,76.59.zarr', '/home/rishabh.mondal/bkdb/districtwise/jhajhar/28.59,76.74.zarr', '/home/rishabh.mondal/bkdb/districtwise/jhajhar/28.45,76.64.zarr', '/home/rishabh.mondal/bkdb/districtwise/jhajhar/28.42,76.65.zarr', '/home/rishabh.mondal/bkdb/districtwise/jhajhar/28.60,76.46.zarr']


In [21]:
files = []

def get_index_and_image(images_path):
    images = []
    index = []

    # for zarr_file in zarr_files:
    zarr_files = [f for f in os.listdir(images_path) if f.endswith(".zarr")]
    for zarr_file in zarr_files:
        image_path = join(images_path, zarr_file)
        # print(image_path)
        files.append(image_path)
        image_ds = xr.open_zarr(image_path, consolidated=False)
        for lat_lag, lon_lag in product(range(-2, 3), repeat=2):
            images.append(torch.tensor(image_ds.sel(lat_lag=lat_lag, lon_lag=lon_lag)['data'].values)[np.newaxis, ...])
            index.append((zarr_file, lat_lag, lon_lag))
    return images


In [22]:
images = get_index_and_image(images_path)


In [23]:
print(images[0].shape)

torch.Size([1, 224, 224, 3])


In [24]:
concatenated_images = torch.cat(images)
concatenated_images= concatenated_images.permute(0,3,2,1)
print(concatenated_images.shape)


torch.Size([42825, 3, 224, 224])


In [25]:
concatenated_images.dtype

torch.uint8

In [26]:

# save_path="/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/tensor_data/west_bengal.pt"
# torch.save({
#     'images': concatenated_images
# }, save_path)

In [28]:
images1=concatenated_images
images1 = images1 / 255
    # mean normalize
images1 = (images1 - images1.mean(dim=(0, 2, 3), keepdim=True)) / images1.std(dim=(0, 2, 3), keepdim=True)
images1.shape


torch.Size([42825, 3, 224, 224])

In [29]:
labels = torch.zeros(len(images1))

testset=TensorDataset(images1,labels)
testloader=DataLoader(testset,batch_size=512,shuffle=False)


In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [31]:
train_model = EfficientNetClassifier(
    models.efficientnet_b0,None, n_classes=2, activation=nn.ReLU(), dropout=0.1
).to(device)

In [32]:
train_model.load_state_dict(torch.load("/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/Saved_model/delhi_whole_data_pretrain_aug.pt"))

<All keys matched successfully>

In [33]:
from torchvision import transforms


In [34]:
# train_model='/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/Saved_model/delhi_whole_data_train_model.pt'
positive_image_indices = []
save_dir = '/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/predicted_positive/jhajhar/'
os.makedirs(save_dir, exist_ok=True)

with torch.no_grad():
    pred_classes =train_model.predict_class(
        dataloader=testloader, batch_size=512, verbose=True
    ).to(device)

zero_predictions = torch.sum(pred_classes == 0)
one_predictions = torch.sum(pred_classes == 1)

print("Number of zero predictions:", zero_predictions.item())
print("Number of one predictions:", one_predictions.item())
images=images1
for i in range(len(images)):
    # Check if the prediction is positive (class 1)
    if pred_classes[i] == 1:
        normalized_image = (images[i] - images[i].min()) / (images[i].max() - images[i].min())
        pil_image = transforms.ToPILImage()(normalized_image.cpu())

        # append the index of the positive image
        positive_image_indices.append(i)
        
        image_name = f"positive_image_{i}.jpg"
        image_path = os.path.join(save_dir, image_name)
        pil_image.save(image_path)            

# Print the list of positive image indices
print(positive_image_indices)
print(len(positive_image_indices))

100%|██████████| 84/84 [11:12<00:00,  8.01s/it]


Number of zero predictions: 42278
Number of one predictions: 547
[26, 27, 38, 41, 43, 44, 80, 82, 328, 378, 500, 515, 650, 651, 672, 687, 896, 914, 1125, 1188, 1199, 1266, 1497, 1775, 1797, 1798, 2349, 2442, 2443, 2507, 2701, 2703, 2708, 2720, 2761, 3194, 3199, 3228, 3245, 3304, 3321, 3324, 3444, 3564, 3627, 3632, 3639, 4076, 4077, 4085, 4092, 4093, 4098, 4187, 4584, 4587, 4591, 4685, 4695, 5283, 5291, 5346, 5614, 5617, 5618, 5624, 5625, 5630, 5637, 5638, 5643, 5962, 5963, 5964, 5968, 5969, 6868, 6893, 6898, 7151, 7152, 7240, 7246, 7247, 7290, 7291, 7293, 7295, 7318, 7375, 7380, 7450, 7457, 7474, 7507, 7603, 7701, 7709, 7712, 7713, 7717, 7722, 7833, 7846, 7990, 7995, 8020, 8055, 8067, 8068, 8074, 8141, 8994, 9036, 9037, 9040, 9194, 9248, 9259, 9393, 9769, 9823, 9824, 9930, 9939, 9948, 9949, 10131, 10204, 10593, 11019, 11024, 11285, 11288, 11290, 11495, 11785, 11790, 11792, 11794, 11799, 11819, 11858, 11861, 11865, 11868, 12030, 12127, 12130, 12131, 12132, 12135, 12142, 12143, 12163, 12